In [ ]:
#pyspark setup

!apt-get update
# Install Java 8 (required by Spark)
!apt-get install openjdk-11-jdk-headless -qq > /dev/null

# Install Spark
!pip install pyspark

# setup environment variables
import os
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/usr/local/lib/python3.10/dist-packages/pyspark"


0% [Working]
            
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]

0% [Connecting to archive.ubuntu.com (185.125.190.82)] [1 InRelease 12.7 kB/129 kB 10%] [Connected t
                                                                                                    
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]

0% [Connecting to archive.ubuntu.com (185.125.190.82)] [1 InRelease 69.2 kB/129 kB 54%] [2 InRelease
0% [Connecting to archive.ubuntu.com (185.125.190.82)] [1 InRelease 95.3 kB/129 kB 74%] [Waiting for
0% [Waiting for headers] [Waiting for headers] [Waiting for headers] [Connecting to ppa.launchpadcon
                                                                                                    
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease

0% [Waiting for headers] [Waiting for headers] [Connecting to ppa.launchpadcontent.net (185.125.190.
            

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date

# Initialize Spark session
spark = SparkSession.builder.appName("Retail Sales Analytics").getOrCreate()

# Load datasets
sales_df = spark.read.csv("/Online Retail .csv", header=True, inferSchema= True) # Corrected the filename
reviews_df = spark.read.csv("/reviews.csv", header=True, inferSchema=True)

# Explore data
sales_df.show(10)
reviews_df.show(10)

# Check schema
sales_df.printSchema()
reviews_df.printSchema()


+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/10 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/10 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/10 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/10 8:26|     4.25|     17850|United

In [ ]:
from pyspark.sql.functions import col

print("Missing Values in Sales Data:")
sales_df.select([col(column).isNull().alias(column) for column in sales_df.columns]).show()

print("Missing Values in Reviews Data:")
reviews_df.select([col(column).isNull().alias(column) for column in reviews_df.columns]).show()


Missing Values in Sales Data:
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-----------+---------+----------+-------+
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|      false|    false|     false|  false|
|    false|    false|      false|   false|    

In [ ]:
sales_df.select([(col(c).isNull().cast("int")).alias(c) for c in sales_df.columns]).summary().show()
reviews_df.select([(col(c).isNull().cast("int")).alias(c) for c in reviews_df.columns]).summary().show()


+-------+---------+---------+--------------------+--------+-----------+---------+-------------------+-------+
|summary|InvoiceNo|StockCode|         Description|Quantity|InvoiceDate|UnitPrice|         CustomerID|Country|
+-------+---------+---------+--------------------+--------+-----------+---------+-------------------+-------+
|  count|   541909|   541909|              541909|  541909|     541909|   541909|             541909| 541909|
|   mean|      0.0|      0.0|0.002683107311375157|     0.0|        0.0|      0.0|  0.249266943342886|    0.0|
| stddev|      0.0|      0.0| 0.05172922949811863|     0.0|        0.0|      0.0|0.43258904241984053|    0.0|
|    min|        0|        0|                   0|       0|          0|        0|                  0|      0|
|    25%|        0|        0|                   0|       0|          0|        0|                  0|      0|
|    50%|        0|        0|                   0|       0|          0|        0|                  0|      0|
|    75%| 

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("Data Cleansing - Simplified Reviews").getOrCreate()

# Load the review dataset
reviews_data_path = "/reviews.csv"  # Replace with the actual file path
reviews_df = spark.read.csv(reviews_data_path, header=True, inferSchema=True)

# Inspect data
print("Initial dataset:")
reviews_df.show(10)
reviews_df.printSchema()

# Handle missing values
print("Handling missing values...")
reviews_df = reviews_df.dropna(subset=["Review", "Sentiment"])

# Debugging step: Check dataset after dropping missing values
print("Dataset after dropping missing values:")
reviews_df.show(10)

# Remove rows with empty or whitespace-only reviewText
print("Filtering non-meaningful text...")
reviews_df = reviews_df.filter(col("Review").rlike(r"\S+"))

# Debugging step: Check dataset after filtering non-meaningful text
print("Dataset after filtering non-meaningful text:")
reviews_df.show(10)

# Validate sentiment values (should be "positive" or "negative")
print("Validating sentiment values...")
valid_sentiments = ["positive", "negative"]
reviews_df = reviews_df.filter(col("Sentiment").isin(valid_sentiments))

# Debugging step: Check dataset after validating sentiment
print("Dataset after validating sentiment:")
reviews_df.show(10)

# Drop duplicates
print("Dropping duplicates...")
reviews_df = reviews_df.dropDuplicates()

# Final dataset preview
print("Final cleaned dataset:")
reviews_df.show(10)

# Save cleaned data for further use
output_path = "cleaned_reviews_data.csv"  # Replace with your desired output path
reviews_df.write.csv(output_path, header=True, mode="overwrite")
print(f"Cleaned dataset saved to {output_path}")


Initial dataset:
+--------------------+---------+
|              Review|Sentiment|
+--------------------+---------+
|This product exce...| Positive|
|The product was d...|  Neutral|
|I had a terrible ...| Negative|
|It's an okay prod...|  Neutral|
|Disappointed with...| Negative|
|Avoid this compan...| Negative|
|I had a terrible ...| Negative|
|Avoid this compan...| Negative|
|This product exce...| Positive|
|This product is o...| Positive|
+--------------------+---------+
only showing top 10 rows

root
 |-- Review: string (nullable = true)
 |-- Sentiment: string (nullable = true)

Handling missing values...
Dataset after dropping missing values:
+--------------------+---------+
|              Review|Sentiment|
+--------------------+---------+
|This product exce...| Positive|
|The product was d...|  Neutral|
|I had a terrible ...| Negative|
|It's an okay prod...|  Neutral|
|Disappointed with...| Negative|
|Avoid this compan...| Negative|
|I had a terrible ...| Negative|
|Avoid this co

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_date, col

spark = SparkSession.builder.appName("Retail Sales Analytics").getOrCreate()

# Set legacy time parser policy to handle the problematic date format
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Load datasets
sales_df = spark.read.csv("/Online Retail .csv", header=True, inferSchema= True) # Corrected the filename


# Inspect data
sales_df.show(10)
sales_df.printSchema()

# Handle missing values
# Drop rows with null CustomerID or Description, as they are critical fields
sales_df = sales_df.dropna(subset=["StockCode", "Description"])

# Drop duplicates
sales_df = sales_df.dropDuplicates()

# Parse InvoiceDate to a proper date format
# Use the correct format based on your data (e.g., "dd/MM/yy HH:mm")
sales_df = sales_df.withColumn("InvoiceDate", to_date(col("InvoiceDate"), "dd/MM/yy HH:mm"))

# Filter out invalid data (e.g., Quantity or UnitPrice less than or equal to 0)
sales_df = sales_df.filter((col("Quantity") > 0) & (col("UnitPrice") > 0))

# Final schema and data preview
sales_df.printSchema()
sales_df.show(10)


+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/10 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/10 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/10 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/10 8:26|     4.25|     17850|United

In [ ]:
# Save cleaned datasets for downstream tasks
sales_df.write.csv("cleaned_sales_data.csv", header=True, mode="overwrite")
reviews_df.write.csv("cleaned_reviews_data.csv", header=True, mode="overwrite")


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, sum, avg, month, year, count, expr

# Initialize Spark session
spark = SparkSession.builder.appName("Sales Aggregation and Feature Engineering").getOrCreate()

# Load the sales dataset
sales_df = spark.read.csv("/content/Online Retail.csv", header=True, inferSchema=True)

# Inspect dataset
print("Initial dataset:")
sales_df.show(10)
sales_df.printSchema()

# Ensure `InvoiceDate` is in proper datetime format
sales_df = sales_df.withColumn("InvoiceDate", col("InvoiceDate").cast("timestamp"))

# Calculate revenue for each transaction
sales_df = sales_df.withColumn("RevenueGenerated", col("Quantity") * col("UnitPrice"))

# Extract month and year for analysis
sales_df = sales_df.withColumn("Year", year(col("InvoiceDate"))).withColumn("Month", month(col("InvoiceDate")))

# Total sales per product per month
print("Calculating total sales per product per month...")
total_sales_per_product = sales_df.groupBy("Year", "Month", "StockCode", "Description") \
    .agg(
        sum("Quantity").alias("TotalQuantitySold"),
        sum("RevenueGenerated").alias("TotalRevenueGenerated")
    ) \
    .orderBy("Year", "Month", "StockCode")

total_sales_per_product.show(10)

# Average revenue per customer segment (Country used as proxy for customer segments)
print("Calculating average revenue per customer segment...")
avg_revenue_per_country = sales_df.groupBy("Country") \
    .agg(avg("RevenueGenerated").alias("AvgRevenuePerCountry"))

avg_revenue_per_country.show()

# Seasonal patterns for top-selling products
print("Analyzing seasonal patterns for top-selling products...")
# Identify top-selling products by total sales
top_products = total_sales_per_product.groupBy("StockCode", "Description") \
    .agg(sum("TotalQuantitySold").alias("TotalSales")) \
    .orderBy(col("TotalSales").desc()) \
    .limit(5)  # Top 5 products

top_products.show()

# Seasonal trends for top products
seasonal_trends = total_sales_per_product.join(top_products, ["StockCode", "Description"]) \
    .groupBy("StockCode", "Description", "Month") \
    .agg(sum("TotalQuantitySold").alias("MonthlySales")) \
    .orderBy("StockCode", "Month")

seasonal_trends.show()

# Feature Engineering
print("Feature engineering...")

# Customer Lifetime Value (CLV): Total revenue generated by each customer
clv = sales_df.groupBy("CustomerID") \
    .agg(sum("RevenueGenerated").alias("CustomerLifetimeValue"))

clv.show(10)

# Product Popularity Score: Weighted by total sales and revenue
product_popularity = sales_df.groupBy("StockCode", "Description") \
    .agg(
        sum("Quantity").alias("TotalUnitsSold"),
        sum("RevenueGenerated").alias("TotalRevenue")
    ) \
    .withColumn("ProductPopularityScore", col("TotalUnitsSold") * col("TotalRevenue"))

product_popularity.show(10)

# Seasonal Trends: Sales aggregated by month across all products
seasonal_trends_all = sales_df.groupBy("Month") \
    .agg(sum("Quantity").alias("TotalMonthlySales")) \
    .orderBy("Month")

seasonal_trends_all.show()

# Save results for further analysis
output_path = "output_sales_aggregation"
total_sales_per_product.write.csv(f"{output_path}/total_sales_per_product", header=True, mode="overwrite")
avg_revenue_per_country.write.csv(f"{output_path}/avg_revenue_per_country", header=True, mode="overwrite")
top_products.write.csv(f"{output_path}/top_products", header=True, mode="overwrite")
seasonal_trends.write.csv(f"{output_path}/seasonal_trends", header=True, mode="overwrite")
clv.write.csv(f"{output_path}/customer_lifetime_value", header=True, mode="overwrite")
product_popularity.write.csv(f"{output_path}/product_popularity", header=True, mode="overwrite")
seasonal_trends_all.write.csv(f"{output_path}/seasonal_trends_all", header=True, mode="overwrite")

print(f"Results saved to {output_path}")


Initial dataset:
+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity| InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|12/1/10 8:26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|12/1/10 8:26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|12/1/10 8:26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|12/1/10 8:26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|12/1/10 8:26|     4.25|

In [ ]:
from pyspark.sql import functions as F
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import RandomForestRegressor
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

# Step 1: Prepare Data
# Seasonal trends data from Task 2
seasonal_trends = sales_df.groupBy("StockCode", "Year", "Month") \
    .agg(
        F.sum("Quantity").alias("TotalQuantitySold"),
        F.sum("RevenueGenerated").alias("TotalRevenueGenerated")
    ) \
    .orderBy("Year", "Month")

# Filter for top products
top_products = seasonal_trends.groupBy("StockCode") \
    .agg(F.sum("TotalQuantitySold").alias("TotalSales")) \
    .orderBy(F.col("TotalSales").desc()) \
    .limit(3)

# Join to filter only top products
filtered_trends = seasonal_trends.join(top_products, on="StockCode", how="inner")

# Create Lag Features
for lag in range(1, 4):  # Create 3 lag features
    filtered_trends = filtered_trends.withColumn(f"lag_{lag}",
                                                 F.lag("TotalQuantitySold", count=lag).over(
                                                     Window.partitionBy("StockCode").orderBy("Year", "Month")))

# Drop rows with null lag values (start of time series)
filtered_trends = filtered_trends.dropna()

# Step 2: Assemble Features
feature_cols = [f"lag_{lag}" for lag in range(1, 4)]  # Lagged features
assembler = VectorAssembler(inputCols=feature_cols, outputCol="features")

# Scale Features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withStd=True, withMean=True)

# Step 3: Train/Test Split
train_data, test_data = filtered_trends.randomSplit([0.8, 0.2], seed=42)

# Step 4: Model Pipeline
rf = RandomForestRegressor(featuresCol="scaled_features", labelCol="TotalQuantitySold", predictionCol="prediction")

pipeline = Pipeline(stages=[assembler, scaler, rf])

# Step 5: Hyperparameter Tuning
param_grid = ParamGridBuilder() \
    .addGrid(rf.numTrees, [10, 20]) \
    .addGrid(rf.maxDepth, [5, 10]) \
    .build()

evaluator = RegressionEvaluator(labelCol="TotalQuantitySold", predictionCol="prediction", metricName="rmse")

cross_validator = CrossValidator(estimator=pipeline, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# Train the Model
cv_model = cross_validator.fit(train_data)

# Step 6: Evaluate the Model
predictions = cv_model.transform(test_data)

# Calculate RMSE and MAE
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})

print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

# Show Predictions
predictions.select("StockCode", "Year", "Month", "TotalQuantitySold", "prediction").show()

# Save the Model
cv_model.bestModel.write().overwrite().save("output_sales_forecasting_model")


TypeError: lag() got an unexpected keyword argument 'count'

TASK 4


In [ ]:
from pyspark.sql import functions as F
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, IDF
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml import Pipeline

# Load Review Dataset
# Assuming `reviews_df` contains columns "text" (review text) and "sentiment" (label: 0 for negative, 1 for positive)
reviews_df = spark.read.csv("/reviews.csv", header=True, inferSchema=True)

# Step 1: Text Preprocessing
# Tokenize the text
tokenizer = Tokenizer(inputCol="text", outputCol="tokens")

# Remove stop words
stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")

# Create n-grams
# Optional: Generate n-grams (bigram as an example)
ngram = F.udf(lambda tokens: [" ".join(tokens[i:i+2]) for i in range(len(tokens)-1)], "array<string>")
reviews_df = reviews_df.withColumn("ngrams", ngram(reviews_df.filtered_tokens))

# Count Vectorizer
vectorizer = CountVectorizer(inputCol="filtered_tokens", outputCol="raw_features")

# TF-IDF Transformation
idf = IDF(inputCol="raw_features", outputCol="features")

# Step 2: Sentiment Model Training
# Logistic Regression Classifier
lr = LogisticRegression(featuresCol="features", labelCol="sentiment", maxIter=10)

# Pipeline for Preprocessing and Model
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, vectorizer, idf, lr])

# Train-Test Split
train_data, test_data = reviews_df.randomSplit([0.8, 0.2], seed=42)

# Fit the Pipeline
model = pipeline.fit(train_data)

# Predictions on Test Data
predictions = model.transform(test_data)

# Evaluate Model
evaluator = MulticlassClassificationEvaluator(labelCol="sentiment", predictionCol="prediction", metricName="accuracy")
accuracy = evaluator.evaluate(predictions)
print(f"Model Accuracy: {accuracy:.2f}")

# Step 3: Sentiment Scoring for Each Product
# Assuming `reviews_df` has an additional `product_id` column
sentiment_scores = predictions.groupBy("product_id") \
    .agg(F.mean(F.col("prediction")).alias("average_sentiment_score"))

# Save Sentiment Scores
sentiment_scores.write.csv("output_sentiment_scores", header=True)

# Show Sentiment Scores
sentiment_scores.show()


AttributeError: 'DataFrame' object has no attribute 'filtered_tokens'

task 3

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, lag
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler, MinMaxScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder
from pyspark.ml.evaluation import RegressionEvaluator



# Initialize SparkSession
spark = SparkSession.builder.appName("TimeSeriesForecasting").getOrCreate()

# Load dataset
file_path = "onlineretail.csv"
data = spark.read.csv(file_path, header=True, inferSchema=True)

# Data preprocessing: Convert date and aggregate data
data = data.withColumn("InvoiceDate", to_date(col("InvoiceDate"), "M/d/yy H:mm"))
aggregated_data = (
    data.groupBy("InvoiceDate", "StockCode")
    .sum("Quantity")
    .withColumnRenamed("sum(Quantity)", "TotalDemand")
    .orderBy("InvoiceDate")
)

# Filter popular product (highest demand)
popular_product = aggregated_data.groupBy("StockCode").sum("TotalDemand").orderBy(col("sum(TotalDemand)").desc()).first()
product_code = popular_product["StockCode"]
filtered_data = aggregated_data.filter(col("StockCode") == product_code).orderBy("InvoiceDate")

# Feature Engineering: Create lag features
window = Window.orderBy("InvoiceDate")
for lag_days in range(1, 4):  # Creating Lag_1, Lag_2, Lag_3
    filtered_data = filtered_data.withColumn(f"Lag_{lag_days}", lag("TotalDemand", lag_days).over(window))

# Drop rows with null values due to lagging
filtered_data = filtered_data.na.drop()

# Assemble features
assembler = VectorAssembler(inputCols=[f"Lag_{i}" for i in range(1, 4)], outputCol="features")
prepared_data = assembler.transform(filtered_data).select("features", col("TotalDemand").alias("label"))

# Split data into training and test sets
train_data, test_data = prepared_data.randomSplit([0.8, 0.2], seed=42)

# Data scaling
scaler = MinMaxScaler(inputCol="features", outputCol="scaledFeatures")
scaler_model = scaler.fit(train_data)
train_data = scaler_model.transform(train_data).select(col("scaledFeatures").alias("features"), "label")
test_data = scaler_model.transform(test_data).select(col("scaledFeatures").alias("features"), "label")

# Train a regression model (Linear Regression used here)
lr = LinearRegression(featuresCol="features", labelCol="label")

# Hyperparameter Tuning with CrossValidator
param_grid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 0.5]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

evaluator = RegressionEvaluator(labelCol="label", predictionCol="prediction", metricName="rmse")

cv = CrossValidator(estimator=lr, estimatorParamMaps=param_grid, evaluator=evaluator, numFolds=3)

# Train model
cv_model = cv.fit(train_data)

# Evaluate model on test data
predictions = cv_model.transform(test_data)
rmse = evaluator.evaluate(predictions, {evaluator.metricName: "rmse"})
mae = evaluator.evaluate(predictions, {evaluator.metricName: "mae"})

# Print evaluation metrics
print(f"Root Mean Squared Error (RMSE): {rmse}")
print(f"Mean Absolute Error (MAE): {mae}")

# Stop SparkSession
spark.stop()


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/12/02 11:17:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/12/02 11:17:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/02 11:17:24 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/02 11:17:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/02 11:17:25 WARN WindowExec: No Partition Defined for Window operation! Moving all data to a single partition, this can cause serious performance degradation.
24/12/02 11:17:25 WARN WindowExec: No Partition Defined for Window 

Root Mean Squared Error (RMSE): 476.81616548715556
Mean Absolute Error (MAE): 209.9168863050334


In [26]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format

# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("OnlineRetail-DemandForecasting") \
    .getOrCreate()

# Set legacy time parser policy to handle datetime parsing
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Step 2: Load the dataset
file_path = "Onlineretail.csv"  # Replace with your file path
data = spark.read.csv(file_path, header=True, inferSchema=True)

# Step 3: Preprocess data
# Convert InvoiceDate to Date type
data = data.withColumn("InvoiceDate", to_date(col("InvoiceDate"), "MM/dd/yy"))

# Add a column for Month-Year for demand aggregation
data = data.withColumn("MonthYear", date_format(col("InvoiceDate"), "yyyy-MM"))

# Aggregate the data to calculate monthly demand for each product
monthly_demand = data.groupBy("MonthYear", "StockCode") \
    .sum("Quantity") \
    .withColumnRenamed("sum(Quantity)", "MonthlyDemand") \
    .orderBy("MonthYear")

# Show aggregated data
monthly_demand.show()

# Step 4: Save the cleaned data for further analysis (optional)
monthly_demand.write.csv("Monthly_Demand.csv", header=True)

print("Processing complete!")


+---------+---------+-------------+
|MonthYear|StockCode|MonthlyDemand|
+---------+---------+-------------+
|  2010-12|    22637|          203|
|  2010-12|    35957|          359|
|  2010-12|    22651|          347|
|  2010-12|   35591T|            3|
|  2010-12|    70007|          418|
|  2010-12|    47421|           72|
|  2010-12|    22906|          212|
|  2010-12|    21447|           22|
|  2010-12|    22572|          261|
|  2010-12|    22170|           71|
|  2010-12|   85131A|           64|
|  2010-12|    21849|           35|
|  2010-12|    47579|            1|
|  2010-12|   85025B|            1|
|  2010-12|   15044C|           10|
|  2010-12|    21943|            6|
|  2010-12|   85036A|            9|
|  2010-12|    22033|           48|
|  2010-12|   84247L|           16|
|  2010-12|    21317|            3|
+---------+---------+-------------+
only showing top 20 rows



Processing complete!


In [31]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("OnlineRetail-DemandForecasting") \
    .getOrCreate()

# Set legacy time parser policy to handle datetime parsing
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Step 2: Load the dataset
file_path = "Onlineretail.csv"  # Replace with your file path
data = spark.read.csv(file_path, header=True, inferSchema=True)

# Step 3: Preprocess data
# Convert InvoiceDate to Date type
data = data.withColumn("InvoiceDate", to_date(col("InvoiceDate"), "MM/dd/yy"))

# Add a column for Month-Year for demand aggregation
data = data.withColumn("MonthYear", date_format(col("InvoiceDate"), "yyyy-MM"))

# Aggregate the data to calculate monthly demand for each product
monthly_demand = data.groupBy("MonthYear", "StockCode") \
    .sum("Quantity") \
    .withColumnRenamed("sum(Quantity)", "MonthlyDemand") \
    .orderBy("MonthYear")

# Step 4: Feature Engineering for Time Series
# Prepare data for machine learning (convert to features and labels)
assembler = VectorAssembler(inputCols=["MonthlyDemand"], outputCol="features")
assembled_data = assembler.transform(monthly_demand)

# Standardize the features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features")
scaled_data = scaler.fit(assembled_data).transform(assembled_data)

# Split the data into training and testing datasets
train_data, test_data = scaled_data.randomSplit([0.8, 0.2], seed=42)

# Step 5: Linear Regression Model
lr = LinearRegression(featuresCol="scaled_features", labelCol="MonthlyDemand", maxIter=10)

# Train the model
lr_model = lr.fit(train_data)

# Evaluate the model on test data
predictions = lr_model.transform(test_data)
evaluator = RegressionEvaluator(labelCol="MonthlyDemand", predictionCol="prediction", metricName="rmse")
rmse = evaluator.evaluate(predictions)
print(f"Root Mean Squared Error (RMSE): {rmse}")

evaluator = RegressionEvaluator(labelCol="MonthlyDemand", predictionCol="prediction", metricName="mae")
mae = evaluator.evaluate(predictions)
print(f"Mean Absolute Error (MAE): {mae}")

# Step 6: Hyperparameter Tuning with CrossValidator
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 0.5]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator,
                          numFolds=3)

# Run cross-validation and select the best model
cv_model = crossval.fit(train_data)
best_model = cv_model.bestModel
print(best_model)

# Evaluate the best model
best_predictions = best_model.transform(test_data)
best_rmse = evaluator.evaluate(best_predictions)
print(f"Best Model RMSE: {best_rmse}")

# Step 7: Save the best model (optional)
# best_model.save("best_demand_forecasting_model")

print("Pipeline execution complete!")


24/12/02 12:14:20 WARN Instrumentation: [0dc8eb75] regParam is zero, which might cause numerical instability and overfitting.


Root Mean Squared Error (RMSE): 8.922915890676034e-12


Mean Absolute Error (MAE): 4.484713280147685e-12


LinearRegressionModel: uid=LinearRegression_775f44d04460, numFeatures=1
Best Model RMSE: 0.004453997358378977
Pipeline execution complete!


In [3]:
pip install ace_tools

Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install ace_tools

Note: you may need to restart the kernel to use updated packages.


In [5]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format, lag
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("DemandForecastingModel") \
    .getOrCreate()

# Set legacy parser policy to handle datetime formats
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Load the dataset
file_path = "Onlineretail.csv"
data = spark.read.csv(file_path, header=True, inferSchema=True)

# Preprocess the data
data = data.withColumn("InvoiceDate", to_date(col("InvoiceDate"), "MM/dd/yy"))

# Filter out invalid data
data = data.filter((col("Quantity") > 0) & (col("UnitPrice") > 0))

# Extract Month-Year and aggregate monthly demand for each product
data = data.withColumn("MonthYear", date_format(col("InvoiceDate"), "yyyy-MM"))
monthly_demand = data.groupBy("MonthYear", "StockCode") \
    .sum("Quantity") \
    .withColumnRenamed("sum(Quantity)", "MonthlyDemand") \
    .orderBy("MonthYear")

# Feature engineering: Adding lag features
window_spec = Window.partitionBy("StockCode").orderBy("MonthYear")
monthly_demand = monthly_demand.withColumn("Lag1", lag("MonthlyDemand", 1).over(window_spec))
monthly_demand = monthly_demand.withColumn("Lag2", lag("MonthlyDemand", 2).over(window_spec))
monthly_demand = monthly_demand.dropna()

# Prepare data for machine learning
assembler = VectorAssembler(inputCols=["Lag1", "Lag2"], outputCol="features")
prepared_data = assembler.transform(monthly_demand)

# Split data into training and testing sets
train_data, test_data = prepared_data.randomSplit([0.8, 0.2], seed=42)

# Scale the features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
scaled_train_data = scaler.fit(train_data).transform(train_data)
scaled_test_data = scaler.fit(train_data).transform(test_data)

# Step 2: Train a Linear Regression model
lr = LinearRegression(featuresCol="scaled_features", labelCol="MonthlyDemand", maxIter=100)

# Fit the model
lr_model = lr.fit(scaled_train_data)

# Evaluate the model
predictions = lr_model.transform(scaled_test_data)
evaluator_rmse = RegressionEvaluator(labelCol="MonthlyDemand", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="MonthlyDemand", predictionCol="prediction", metricName="mae")

rmse = evaluator_rmse.evaluate(predictions)
mae = evaluator_mae.evaluate(predictions)

# Hyperparameter tuning with CrossValidator
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 0.5]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator_rmse,
                          numFolds=3)

cv_model = crossval.fit(scaled_train_data)
best_model = cv_model.bestModel

# Evaluate the best model
best_predictions = best_model.transform(scaled_test_data)
best_rmse = evaluator_rmse.evaluate(best_predictions)
best_mae = evaluator_mae.evaluate(best_predictions)

# Save results for reference
results = {
    "Initial RMSE": rmse,
    "Initial MAE": mae,
    "Best RMSE": best_rmse,
    "Best MAE": best_mae
}

import pandas as pd
results_df = pd.DataFrame([results])

print(results_df)



24/12/02 12:42:58 WARN Instrumentation: [0c925972] regParam is zero, which might cause numerical instability and overfitting.


   Initial RMSE  Initial MAE   Best RMSE   Best MAE
0    270.171969   115.989885  270.218604  116.07705


In [9]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_date, date_format, lag
from pyspark.sql.window import Window
from pyspark.ml.feature import VectorAssembler, StandardScaler
from pyspark.ml.regression import LinearRegression
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("DemandForecastingModel") \
    .getOrCreate()

# Set legacy parser policy to handle datetime formats
spark.conf.set("spark.sql.legacy.timeParserPolicy", "LEGACY")

# Load the dataset
file_path = "Onlineretail.csv"
data = spark.read.csv(file_path, header=True, inferSchema=True)

# Preprocess the data
data = data.withColumn("InvoiceDate", to_date(col("InvoiceDate"), "MM/dd/yy"))

# Filter out invalid data
data = data.filter((col("Quantity") > 0) & (col("UnitPrice") > 0))

# Extract Month-Year and aggregate monthly demand for each product
data = data.withColumn("MonthYear", date_format(col("InvoiceDate"), "yyyy-MM"))
monthly_demand = data.groupBy("MonthYear", "StockCode") \
    .sum("Quantity") \
    .withColumnRenamed("sum(Quantity)", "MonthlyDemand") \
    .orderBy("MonthYear")

# Feature engineering: Adding lag features
window_spec = Window.partitionBy("StockCode").orderBy("MonthYear")
monthly_demand = monthly_demand.withColumn("Lag1", lag("MonthlyDemand", 1).over(window_spec))
monthly_demand = monthly_demand.withColumn("Lag2", lag("MonthlyDemand", 2).over(window_spec))
monthly_demand = monthly_demand.dropna()

# Prepare data for machine learning
assembler = VectorAssembler(inputCols=["Lag1", "Lag2"], outputCol="features")
prepared_data = assembler.transform(monthly_demand)

# Split data into training and testing sets
train_data, test_data = prepared_data.randomSplit([0.8, 0.2], seed=42)

# Scale the features
scaler = StandardScaler(inputCol="features", outputCol="scaled_features", withMean=True, withStd=True)
scaled_train_data = scaler.fit(train_data).transform(train_data)
scaled_test_data = scaler.fit(train_data).transform(test_data)

# Step 2: Train a Linear Regression model
lr = LinearRegression(featuresCol="scaled_features", labelCol="MonthlyDemand", maxIter=100)

# Fit the model
lr_model = lr.fit(scaled_train_data)

# Evaluate the model
predictions = lr_model.transform(scaled_test_data)
evaluator_rmse = RegressionEvaluator(labelCol="MonthlyDemand", predictionCol="prediction", metricName="rmse")
evaluator_mae = RegressionEvaluator(labelCol="MonthlyDemand", predictionCol="prediction", metricName="mae")

rmse = evaluator_rmse.evaluate(predictions)
mae = evaluator_mae.evaluate(predictions)

# Hyperparameter tuning with CrossValidator
paramGrid = ParamGridBuilder() \
    .addGrid(lr.regParam, [0.01, 0.1, 0.5]) \
    .addGrid(lr.elasticNetParam, [0.0, 0.5, 1.0]) \
    .build()

crossval = CrossValidator(estimator=lr,
                          estimatorParamMaps=paramGrid,
                          evaluator=evaluator_rmse,
                          numFolds=3)

cv_model = crossval.fit(scaled_train_data)
best_model = cv_model.bestModel

# Evaluate the best model
best_predictions = best_model.transform(scaled_test_data)
best_rmse = evaluator_rmse.evaluate(best_predictions)
best_mae = evaluator_mae.evaluate(best_predictions)

# Save results for reference
results = {
    "Initial RMSE": rmse,
    "Initial MAE": mae,
    "Best RMSE": best_rmse,
    "Best MAE": best_mae
}

# Display results
print("Model Evaluation Results:")
print(f"Initial RMSE: {rmse}")
print(f"Initial MAE: {mae}")
print(f"Best RMSE after Tuning: {best_rmse}")
print(f"Best MAE after Tuning: {best_mae}")


24/12/02 12:45:01 WARN Instrumentation: [96dee6af] regParam is zero, which might cause numerical instability and overfitting.


Model Evaluation Results:
Initial RMSE: 270.1719690365431
Initial MAE: 115.98988502326142
Best RMSE after Tuning: 270.2186044916398
Best MAE after Tuning: 116.07705002550536


task 4

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, udf, when, lit
from pyspark.ml.feature import Tokenizer, StopWordsRemover, CountVectorizer, NGram
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.types import StringType
from pyspark.sql.functions import avg

# Step 1: Initialize Spark Session
spark = SparkSession.builder \
    .appName("SentimentAnalysis") \
    .getOrCreate()

# Step 2: Load Dataset
data = spark.read.csv("reviews.csv", header=True, inferSchema=True)

# Ensure required columns
print("Dataset Columns:", data.columns)

# Add 'Product' column if missing
if "Product" not in data.columns:
    data = data.withColumn("Product", lit("DefaultProduct"))

# Step 3: Text Preprocessing
tokenizer = Tokenizer(inputCol="Review", outputCol="tokens")
stopwords_remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens")
ngram = NGram(n=2, inputCol="filtered_tokens", outputCol="ngrams")
vectorizer = CountVectorizer(inputCol="ngrams", outputCol="features")

# Step 4: Label Encoding
label_indexer = StringIndexer(inputCol="Sentiment", outputCol="label")

# Step 5: Train-Test Split
train_data, test_data = data.randomSplit([0.8, 0.2], seed=42)

# Step 6: Model Training (Logistic Regression)
lr = LogisticRegression(featuresCol="features", labelCol="label")

# Pipeline
pipeline = Pipeline(stages=[tokenizer, stopwords_remover, ngram, vectorizer, label_indexer, lr])

# Fit model
model = pipeline.fit(train_data)

# Step 7: Evaluate Model
predictions = model.transform(test_data)
correct_predictions = predictions.filter(col("label") == col("prediction")).count()
accuracy = correct_predictions / test_data.count()
print(f"Model Accuracy: {accuracy:.2f}")

# Step 8: Map Predictions to Sentiment Labels
def map_label_to_sentiment(pred):
    return "Positive" if pred == 1.0 else "Negative" if pred == 0.0 else "Neutral"

map_udf = udf(map_label_to_sentiment, StringType())
predictions = predictions.withColumn("PredictedSentiment", map_udf(col("prediction")))

# Step 9: Sentiment Scoring
sentiment_score = predictions.groupBy("Product").agg(
    avg(when(col("PredictedSentiment") == "Positive", 1)
        .when(col("PredictedSentiment") == "Negative", -1)
        .otherwise(0)).alias("SentimentScore")
)

# Step 10: Show Results
sentiment_score.show()

# Step 11: Save Model and Results
model.write().overwrite().save("sentiment_analysis_model")
sentiment_score.write.csv("sentiment_score.csv", header=True)


: 

task5